In [36]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import (
    DecisionTreeClassifier,
    GBTClassifier,
    RandomForestClassifier,
)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Crear la sesión de Spark
spark = SparkSession.builder.appName("Iris Classification").getOrCreate()


In [37]:
# Cargar el dataset CSV
data = spark.read.csv("iris.csv", header=True, inferSchema=True)

# Mostrar las primeras filas del dataset para comprobar la estructura
data.show(5)

+-----------------+----------------+-----------------+----------------+-------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|species|
+-----------------+----------------+-----------------+----------------+-------+
|              5.1|             3.5|              1.4|             0.2|      0|
|              4.9|             3.0|              1.4|             0.2|      0|
|              4.7|             3.2|              1.3|             0.2|      0|
|              4.6|             3.1|              1.5|             0.2|      0|
|              5.0|             3.6|              1.4|             0.2|      0|
+-----------------+----------------+-----------------+----------------+-------+
only showing top 5 rows



In [38]:
# Filtrar el dataset para incluir solo dos clases (0 y 1) para clasificación binaria
binary_data = data.filter(data["species"].isin(0, 1))

# Mostrar las primeras filas del dataset filtrado
binary_data.show(5)


+-----------------+----------------+-----------------+----------------+-------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|species|
+-----------------+----------------+-----------------+----------------+-------+
|              5.1|             3.5|              1.4|             0.2|      0|
|              4.9|             3.0|              1.4|             0.2|      0|
|              4.7|             3.2|              1.3|             0.2|      0|
|              4.6|             3.1|              1.5|             0.2|      0|
|              5.0|             3.6|              1.4|             0.2|      0|
+-----------------+----------------+-----------------+----------------+-------+
only showing top 5 rows



In [39]:
# Transformar en vector los datos numéricos usando VectorAssembler
# Definir las columnas de características
feature_columns = [
    "sepal length (cm)",
    "sepal width (cm)",
    "petal length (cm)",
    "petal width (cm)",
]

# Crear el VectorAssembler para las características
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transformar las características y agregar la columna 'features' al DataFrame
binary_data = assembler.transform(binary_data)

In [40]:
# Transformar los datos correspondientes a la columna especies usando StringIndexer
# Crear el StringIndexer para la columna 'species'
indexer = StringIndexer(inputCol="species", outputCol="label")

# Ajustar el indexador en el conjunto de datos filtrado
binary_data = indexer.fit(binary_data).transform(binary_data)

# Convertir la columna 'label' a tipo Double (necesario para GBTClassifier)
from pyspark.sql.types import DoubleType

binary_data = binary_data.withColumn("label", binary_data["label"].cast(DoubleType()))

# Mostrar las primeras filas para verificar las nuevas columnas
binary_data.select("species", "label", "features").show(5)

+-------+-----+-----------------+
|species|label|         features|
+-------+-----+-----------------+
|      0|  0.0|[5.1,3.5,1.4,0.2]|
|      0|  0.0|[4.9,3.0,1.4,0.2]|
|      0|  0.0|[4.7,3.2,1.3,0.2]|
|      0|  0.0|[4.6,3.1,1.5,0.2]|
|      0|  0.0|[5.0,3.6,1.4,0.2]|
+-------+-----+-----------------+
only showing top 5 rows



In [41]:
# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data = binary_data.randomSplit([0.8, 0.2], seed=42)

# Verificar la distribución de clases en el conjunto de entrenamiento
train_data.groupBy("label").count().show()

# %%
# Evaluador de clasificación
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)

+-----+-----+
|label|count|
+-----+-----+
|  0.0|   38|
|  1.0|   45|
+-----+-----+



In [42]:
# Calcular la predicción y precisión usando Decision Tree Classifier
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")

# Entrenar el modelo
dt_model = dt.fit(train_data)

# Realizar predicciones sobre los datos de prueba
dt_predictions = dt_model.transform(test_data)

# Evaluar la precisión del modelo
dt_accuracy = evaluator.evaluate(dt_predictions)

# Mostrar los resultados
print(f"Precisión del modelo Decision Tree: {dt_accuracy:.4f}")

Precisión del modelo Decision Tree: 1.0000


In [43]:
# Calcular la predicción y precisión usando Gradient-Boosted Tree Classifier
gbt = GBTClassifier(featuresCol="features", labelCol="label", maxIter=10)

# Entrenar el modelo
gbt_model = gbt.fit(train_data)

# Realizar predicciones sobre los datos de prueba
gbt_predictions = gbt_model.transform(test_data)

# Evaluar la precisión del modelo
gbt_accuracy = evaluator.evaluate(gbt_predictions)

# Mostrar los resultados
print(f"Precisión del modelo Gradient-Boosted Tree: {gbt_accuracy:.4f}")


Precisión del modelo Gradient-Boosted Tree: 1.0000


In [44]:
# Calcular la predicción y precisión usando Random Forest Classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100)

# Entrenar el modelo
rf_model = rf.fit(train_data)

# Realizar predicciones sobre los datos de prueba
rf_predictions = rf_model.transform(test_data)

# Evaluar la precisión del modelo
rf_accuracy = evaluator.evaluate(rf_predictions)

# Mostrar los resultados
print(f"Precisión del modelo Random Forest: {rf_accuracy:.4f}")

Precisión del modelo Random Forest: 1.0000


In [45]:
# Finalizar la sesión de Spark
spark.stop()